In [0]:
!pip install -q torch torchvision numpy

import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
from collections import OrderedDict
import itertools


class Adder(nn.Module):
  def __init__(self, model, thres = 0.4, function = "var"):
        super(Adder, self).__init__()
        self.model = model
        self.to_add = {}
        self.next_layer = {}
        self.thres = thres
        self.to_train = True #true is train, false is test
        
        layer_names = list(model.pruning_layers._modules.keys())
        for i, layer_name in enumerate(layer_names):
            layer = model.pruning_layers._modules.get(layer_name)
            if(i < len(model.pruning_layers._modules)-1):
                layer.layer_name = layer_name
                self.next_layer[layer_name] = \
                    model.pruning_layers._modules.get(layer_names[i+1])
                if(function == "var" ): #and not to_train
                    layer.register_forward_hook(self.low_variance_nodes)
            i += 1
          
          
  def add(self):
      i = 0
      for layer_name, seq_layer in model.pruning_layers._modules.items():
        if(i < len(model.pruning_layers._modules)-1):
          layer = seq_layer._modules['0']
          #print("adding nodes ", layer_name, " ", self.to_add)
          nodes_to_add = self.to_add[layer_name]
          np_weights = layer.weight.data.cpu().numpy()

          np_weights[nodes_to_add,:, :, :] = np.random.normal(np_weights[nodes_to_add,:, :, :], 0.5*np.std(np_weights[nodes_to_add,:, :, :]))
          weights_to_add = np_weights[nodes_to_add,:, :, :]
          weights_to_add = np.random.normal(weights_to_add, 0.5*np.std(weights_to_add))
          #weights_to_add = 1/10*np.random.randn(weights_to_add.shape[0],weights_to_add.shape[1],weights_to_add.shape[2],weights_to_add.shape[3])
          np_weights = np.concatenate((np_weights, weights_to_add), axis = 0).astype(np.float)
          np_weights = np.random.normal(np_weights,0.5*np.std(np_weights)) 
          layer.weight = Parameter(torch.from_numpy(np_weights).type(torch.FloatTensor).cuda())
          
          num_nodes_to_add = len(nodes_to_add)
          # increment out features
          layer.out_channels += num_nodes_to_add
          
          # increment in features of next layer
          next_layer = self.next_layer[layer_name]
          next_layer = next_layer._modules['0']
          next_layer.in_channels += num_nodes_to_add
          
          bias_weights = layer.bias.data.cpu().numpy()
          bias_weights[nodes_to_add] = np.random.normal(bias_weights[nodes_to_add],0.5*np.std(bias_weights[nodes_to_add])) 
          bias_to_add = bias_weights[nodes_to_add]
          bias_to_add = np.random.normal(bias_to_add,0.5*np.std(bias_to_add)) 
          #bias_to_add = 1/10*np.random.randn(bias_to_add.shape[0])
          bias_weights = np.concatenate((bias_weights, bias_to_add)).astype(np.float)
          bias_weights = np.random.normal(bias_weights,0.5*np.std(bias_weights)) 
          layer.bias = Parameter(torch.from_numpy(bias_weights).type(torch.FloatTensor).cuda())
          
          # add dimension to kernels
          np_weights = next_layer.weight.data.cpu().numpy()
          np_weights = np.concatenate((np_weights, np_weights[:, nodes_to_add, :, :]), axis = 1)
          next_layer.weight = Parameter(torch.from_numpy(np_weights).cuda())  
          
          #update batch layer
          batch_layer = seq_layer._modules['3']
         
          running_mean = batch_layer.running_mean.data.cpu().numpy()
          running_mean = np.concatenate((running_mean, running_mean[nodes_to_add]))
          batch_layer.running_mean = torch.from_numpy(running_mean).cuda()
          
          batch_weight = batch_layer.weight.data.cpu().numpy()
          batch_weight = np.concatenate((batch_weight, batch_weight[nodes_to_add]))
          batch_layer.weight = Parameter(torch.from_numpy(batch_weight).cuda())
          
          batch_bias = batch_layer.bias.data.cpu().numpy()
          batch_bias = np.concatenate((batch_bias, batch_bias[nodes_to_add]))
          batch_layer.bias = Parameter(torch.from_numpy(batch_bias).cuda())
          
          running_var = batch_layer.running_var.data.cpu().numpy()
          running_var = np.concatenate((running_var, running_var[nodes_to_add]))
          batch_layer.running_var = torch.from_numpy(running_var).cuda()
          
        i+=1
       
          
  def forward(self, x):
        return self.model(x)
          
    
  def low_variance_nodes(self, layer, input, output):
      if(not self.to_train):
        h, w = output.shape[2], output.shape[3]
        # get correlations
        layer_vars = np.apply_over_axes(np.var, output.cpu().detach().numpy(), [0,2,3])
        #print("average layer var", np.mean(layer_vars),"min layer var", np.min(layer_vars), layer_vars)
        layer_vars = np.abs(layer_vars)
        self.to_add[layer.layer_name] = np.where((layer_vars.ravel() < self.thres))[0]
        
            
class Pruner(nn.Module):
    def __init__(self, model, thres = 0.95, function="corrs"):
        super(Pruner, self).__init__()
        self.model = model
        self.to_remove = {}
        self.next_layer = {}
        self.thres = thres
        self.to_train = True #true is train, false is test

        layer_names = list(model.pruning_layers._modules.keys())
        for i, layer_name in enumerate(layer_names):
            layer = model.pruning_layers._modules.get(layer_name)
            if(i < len(model.pruning_layers._modules)-1):
                layer.layer_name = layer_name
                self.next_layer[layer_name] = \
                    model.pruning_layers._modules.get(layer_names[i+1])
                if(function == "corrs"):
                    layer.register_forward_hook(self.correlations)
                elif(function == "l1"):
                    layer.register_forward_hook(self.min_L1)
            i += 1
    
    def prune(self):
        i = 0
        for layer_name, seq_layer in model.pruning_layers._modules.items():
            if(i < len(model.pruning_layers._modules)-1):
                print("remove from ", layer_name, " ", self.to_remove)
                nodes_to_remove = self.to_remove[layer_name]
                n_remove = len(nodes_to_remove)

                layer = seq_layer._modules['0']
                layer.out_channels -= n_remove

                # delete layer_index row in layer, and column in next layer
                np_weights = layer.weight.data.cpu().numpy()
                np_weights = np.delete(np_weights, nodes_to_remove, axis=0)
                layer.weight = Parameter(torch.from_numpy(np_weights).cuda())

                layer_weights = layer.bias.data.cpu().numpy()
                layer_weights = np.delete(layer_weights, nodes_to_remove)
                layer.bias = Parameter(torch.from_numpy(layer_weights).cuda())

                next_layer = self.next_layer[layer_name]._modules['0']
                next_layer.in_channels -= 1
                np_weights = next_layer.weight.data.cpu().numpy()
                np_weights = np.delete(np_weights, nodes_to_remove, axis=1)
                next_layer.weight = Parameter(torch.from_numpy(np_weights).cuda())
            i += 1
    
    
    def forward(self, x):
        return self.model(x)
    
    def pairs_of_low_correlations(self, layer, input, output):
      if(not self.to_train):
        h, w = output.shape[2], output.shape[3]
        # get correlations
        n_filters = output.shape[1]
        corrs = np.zeros((n_filters, n_filters))
        for i in range(h):
            for j in range(w):
                ap = output[:,:,i,j]
                corrs += np.corrcoef(ap.detach().cpu().numpy().T)
        corrs /= n_filters

        # find filter pairs above correlation threshold
        corrs = np.abs(corrs)
        np.fill_diagonal(corrs, 0)
        sorted_corr = np.sort(corrs.ravel())

        nodes_to_prune = np.where((self.thres <= corrs[:,:]))
        rows, cols = nodes_to_prune[0], nodes_to_prune[1]

        # get filters to remove
        for r in range(0, len(rows)):
            r_i = rows[r]
            if(r_i > -1):
              ind = np.where(cols[r] == rows[:])
              rows[ind] = -1
              ind = np.where(rows[r] == rows[r+1:])
              rows[ind] = -1
        self.to_remove[layer.layer_name] = rows[np.where(rows[:] > -1)]
    
        
    '''
    PRUNING FUNCTIONS
    START HERE
    '''
    def min_L1(self, layer, input, output):
      if(not self.to_train):
        activs = np.abs(input[0].numpy())
        L1 = np.apply_over_axes(np.sum, activs, [0,2,3])
        L1 = L1.reshape(L1.shape[1])
        self.to_remove[layer.layer_name] = [np.argmin(L1)]

    def correlations(self, layer, input, output):
      if(not self.to_train):
        h, w = output.shape[2], output.shape[3]
        # get correlations
        n_filters = output.shape[1]
        corrs = np.zeros((n_filters, n_filters))
        for i in range(h):
            for j in range(w):
                ap = output[:,:,i,j]
                corrs += np.corrcoef(ap.detach().cpu().numpy().T)
        corrs /= n_filters

        # find filter pairs above correlation threshold
        corrs = np.abs(corrs)
        np.fill_diagonal(corrs, 0)
        sorted_corr = np.sort(corrs.ravel())

        nodes_to_prune = np.where((self.thres <= corrs[:,:]))
        rows, cols = nodes_to_prune[0], nodes_to_prune[1]

        # get filters to remove
        for r in range(0, len(rows)):
            r_i = rows[r]
            if(r_i > -1):
              ind = np.where(cols[r] == rows[:])
              rows[ind] = -1
              ind = np.where(rows[r] == rows[r+1:])
              rows[ind] = -1
        self.to_remove[layer.layer_name] = rows[np.where(rows[:] > -1)]


In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.pruning_layers = nn.Sequential(OrderedDict([
            ("Layer1",
                nn.Sequential(
                  nn.Conv2d(3, 64, 5, padding=2),
                  nn.MaxPool2d(2),
                  nn.ReLU(), 
                  nn.BatchNorm2d(64, track_running_stats=True)
                )),
            ("Layer2",
                nn.Sequential(
                  nn.Conv2d(64, 128, 5, padding=2),
                  nn.MaxPool2d(2),
                  nn.ReLU(), 
                  nn.BatchNorm2d(128, track_running_stats=True)
                )),
            ("Layer4",
                nn.Sequential(
                  nn.Conv2d(128, 256, 5, padding=2),
                  nn.MaxPool2d(2),
                  nn.ReLU(), 
                  nn.BatchNorm2d(256, track_running_stats=True)
                )),
            ("Layer5",
                nn.Sequential(
                  nn.Conv2d(256, 256, 5, padding=2),
                  nn.MaxPool2d(2),
                  nn.ReLU(), 
                  nn.BatchNorm2d(256, track_running_stats=True),
                  nn.Dropout(0.5)
                ))
            ])) 
        self.fc1 = nn.Linear(1024, 10)
        self.fc2 = nn.Linear(10, 10)

    def forward(self, x):
        #print("in forward size of x ", x.shape)
        x = self.pruning_layers(x)
        x = x.view(x.size(0),-1)
        #print(x.size())
        x = self.fc1(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = Net()

#pruning_model = Pruner(model, thres = 0.9, function = "corrs")

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            #data.unsqueeze_(0)
            #print("in test size of data ", data.shape)
            output = model(data)
            #print("error")
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    return 1. * correct / len(test_loader.dataset)

torch.manual_seed(1)

device = torch.device("cuda")

kwargs = {'num_workers': 1, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True, **kwargs)

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)#, momentum=0.5)

Files already downloaded and verified


In [0]:
import time

epoch_range = 5

for epoch in range(epoch_range):
  train(model, device, train_loader, optimizer, epoch)
  test(model, device, test_loader)

val_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True, **kwargs)

pruning_model = Adder(model, thres = 0.5, function = "var")

pruning_model.to_train = False
t0 = time.time()
acc = test(pruning_model, device, val_loader)
t1 = time.time()
print("testing time", (t1-t0))


Train Epoch: 0 [0/50000 (0%)]	Loss: 2.341164
Train Epoch: 0 [6400/50000 (13%)]	Loss: 1.489525
Train Epoch: 0 [12800/50000 (26%)]	Loss: 1.231428
Train Epoch: 0 [19200/50000 (38%)]	Loss: 1.090029
Train Epoch: 0 [25600/50000 (51%)]	Loss: 1.114900
Train Epoch: 0 [32000/50000 (64%)]	Loss: 1.134089
Train Epoch: 0 [38400/50000 (77%)]	Loss: 1.001651
Train Epoch: 0 [44800/50000 (90%)]	Loss: 0.913920

Test set: Average loss: 1.3661, Accuracy: 5617/10000 (56%)

Train Epoch: 1 [0/50000 (0%)]	Loss: 0.624488
Train Epoch: 1 [6400/50000 (13%)]	Loss: 0.744100
Train Epoch: 1 [12800/50000 (26%)]	Loss: 0.960869
Train Epoch: 1 [19200/50000 (38%)]	Loss: 0.753951
Train Epoch: 1 [25600/50000 (51%)]	Loss: 0.656955
Train Epoch: 1 [32000/50000 (64%)]	Loss: 0.703311
Train Epoch: 1 [38400/50000 (77%)]	Loss: 0.774766
Train Epoch: 1 [44800/50000 (90%)]	Loss: 0.660900

Test set: Average loss: 0.7522, Accuracy: 7438/10000 (74%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 0.507769
Train Epoch: 2 [6400/50000 (13%)]	Loss: 0.516

In [0]:
def test_acc(model):
  val_loader = torch.utils.data.DataLoader(
      datasets.CIFAR10('../data', train=False, transform=transforms.Compose([
                         transforms.ToTensor(),
                         transforms.Normalize((0.1307,), (0.3081,))
                     ])),
      batch_size=64, shuffle=True, **kwargs)
  model.to_train = False
  t0 = time.time()
  acc = test(model, device, val_loader)
  t1 = time.time()
  model.to_train = True
  print("Inference time ", t1 - t0)
  print("Accuracy", acc)
  return acc

def retrain(model):
  for epoch in range(epoch_range):
      model.to_train = True 
      train(model, device, train_loader, optimizer, epoch)
      model.to_train = False 
      test(model, device, test_loader)
      model.to_train = True
  return test_acc(model)

def prune_loop(model, thresholds, sacrifice):
  init_model_acc = test_acc(model)

  for thres in thresholds:
    print("***** THRES = ", thres, " *****)")
    new_model = Pruner(model, thres = thres, function = "corrs")
    new_model.to_train = False
    acc = test(new_model, device, val_loader)
    new_model.prune()
    new_model.to_train = True
    new_model_acc = retrain(new_model)
    
    i = 0
    while(new_model_acc <= init_model_acc - sacrifice and i < 3):
      print("--- accuracy drop ", i, " ---")
      new_model_acc = retrain(new_model)
      i += 1

    if(new_model_acc <= init_model_acc - sacrifice):
      return model
    model = new_model.model
    print("Number of parameters", sum(p.numel() for p in model.parameters()))
  return model

def add_loop(model,thresholds, sacrifice):
  init_model_acc = test_acc(model)

  for thres in thresholds:
    print("***** THRES = ", thres, " *****)")
    new_model = Adder(model, thres = thres, function = "var")
    new_model.to_train = False
    acc = test(new_model, device, val_loader)
    new_model.add()
    print(new_model)
    new_model.to_train = True
    new_model_acc = retrain(new_model)
    
    i = 0
    while(new_model_acc <= init_model_acc - sacrifice and i < 3):
      print("--- accuracy drop ", i, " ---")
      new_model_acc = retrain(new_model)
      i += 1

    if(new_model_acc <= init_model_acc - sacrifice):
      return model
    model = new_model.model
    print("Number of parameters", sum(p.numel() for p in model.parameters()))
  return model
  
sacrifice = 0.01
thresholds = [.001, .02, .06, .1]
new_model = add_loop(model, thresholds, sacrifice)
    


Test set: Average loss: 0.7188, Accuracy: 7760/10000 (78%)

('Inference time ', 3.83154296875)
('Accuracy', 0.776)
('***** THRES = ', 0.001, ' *****)')

Test set: Average loss: 0.7188, Accuracy: 7760/10000 (78%)

Adder(
  (model): Net(
    (pruning_layers): Sequential(
      (Layer1): Sequential(
        (0): Conv2d(3, 113, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (2): ReLU()
        (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (Layer2): Sequential(
        (0): Conv2d(113, 198, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (2): ReLU()
        (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (Layer4): Sequential(
        (0): Conv2d(198, 376, kernel_size=(5, 5), stride=(1, 1), p